In [35]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [36]:
genres_data = pd.read_csv("./dataset/top_genres_in_the_world.csv")


genres_columns = ['Hip hop/Rap/R&b', 'EDM', 'Pop', 'Rock/Metal', 'Latin/Reggaeton', 'Other']
genres_data_subset = genres_data[genres_columns]
row_sums = genres_data_subset.sum(axis=1)
normalized_genres_data = genres_data_subset.div(row_sums, axis=0).mul(100, axis=0)
normalized_genres_data['Country'] = genres_data['Country']
countries_and_pop = normalized_genres_data[['Country', 'Pop']]

# Create the world map plot using Plotly
fig1_pop = px.choropleth(countries_and_pop, 
                    locations="Country", 
                    locationmode="country names", 
                    color="Pop", 
                    range_color=[0, 100],
                    title="Popularity of Pop Music Genres in the World"
                   )
fig1_pop.update_layout(
    # height=400,
    title_font=dict(size=25),
    height=600,
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)

fig1_pop.show()

In [37]:
countries_and_edm = normalized_genres_data[['Country', 'EDM']]

# Create the world map plot using Plotly
fig2_edm = px.choropleth(countries_and_edm, 
                    locations="Country", 
                    locationmode="country names", 
                    color="EDM", 
                    range_color=[0, 100],
                    title="Popularity of EDM Music Genres in the World"
                   )
fig2_edm.update_layout(
    # height=400,
    title_font=dict(size=25),
    height=600,
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)

fig2_edm.show()

In [38]:
countries_and_rock = normalized_genres_data[['Country', 'Rock/Metal']]

# Create the world map plot using Plotly
fig3_rock = px.choropleth(countries_and_rock, 
                    locations="Country", 
                    locationmode="country names", 
                    color='Rock/Metal', 
                    range_color=[0, 100],
                    title="Popularity of Rock/Metal Music Genres in the World"
                   )
fig3_rock.update_layout(
    title_font=dict(size=25),
    height=600,
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)

fig3_rock.show()

In [39]:
countries_and_hiphop = normalized_genres_data[['Country', 'Hip hop/Rap/R&b']]

# Create the world map plot using Plotly
fig4_hiphop = px.choropleth(countries_and_hiphop, 
                    locations="Country", 
                    locationmode="country names", 
                    color='Hip hop/Rap/R&b', 
                    range_color=[0, 100],
                    title="Popularity of 'Hip hop/Rap/R&B' Music in the World"
                   )
fig4_hiphop.update_layout(
    title_font=dict(size=25),
    height=600,
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)

fig4_hiphop.show()

In [40]:
path_dataset2 = "./dataset/albumlist.csv"
dataset = pd.read_csv(path_dataset2)

In [41]:
genres_list = dataset['Genre'].value_counts()
count_map = {}
for genres in genres_list.keys():
    count = genres_list[genres]
    genres = genres.strip()
    split_genres = genres.split(',')
    for genre in split_genres:
        genre = genre.strip(" & ")
        if genre in count_map.keys():
            count_map[genre] = count_map[genre] + count
        else:
            count_map[genre] = count
count_map = dict(sorted(count_map.items(), key=lambda item: item[1], reverse=True))

In [42]:
import plotly.express as px

# Create a new DataFrame to store the data for the plot
data = []

# Specify the order of genres
all_genres = ["Rock","Funk / Soul","Pop","Country","Folk","Electronic","Latin"]

# Loop over each decade
for start_year in range(1960, 2001, 10):
    end_year = start_year + 9
    filtered_dataset = dataset[(dataset['Year'] >= start_year) & (dataset['Year'] <= end_year)]
    genre_albums = {genre: 0 for genre in all_genres}
    counts = filtered_dataset['Genre'].value_counts()
    for genres in counts.index:
        count = counts[genres]
        genres = genres.strip()
        split_genres = genres.split(',')
        for genre in split_genres:
            genre = genre.strip(" & ")
            if genre in genre_albums:
                genre_albums[genre] += count
    for genre, count in genre_albums.items():
        data.append({'Decade': str(start_year) + 's', 'Genre': genre, 'Number of Albums': count})

# Convert the data to a DataFrame
df = pd.DataFrame(data)
df = df[df['Genre'].isin(all_genres)]
# Create the animated bar chart
fig5_genre_list = px.bar(df, x='Number of Albums', y='Genre', animation_frame='Decade',orientation='h', 
             range_x=[0, df['Number of Albums'].max()], 
             category_orders={'Genre': all_genres})

fig5_genre_list.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500

# Show the plot
fig5_genre_list.show()

In [43]:
#figure on grammy album winners from different countries

# First we generated album_winner_grammy_countries.csv from grammys-best album.csv by getting
# latitudes and longitudes of cities to which the artists belong

# path_dataset2 = "./dataset/grammys-best album.csv"
# dataset = pd.read_csv(path_dataset2)
# def year_to_decade(year):
#     return str(year // 10 * 10) + "s"

# dataset['decade'] = dataset['Year'].apply(year_to_decade)
# grouped_data = dataset.groupby(['decade', 'City/town of birth/origin']).size().reset_index(name='count')
# def get_lat_long(city):
#     location = geolocator.geocode(city)
#     if location:
#         return location.latitude, location.longitude
#     else:
#         return None, None
    
# grouped_data['latitude'], grouped_data['longitude'] = zip(*grouped_data['City/town of birth/origin'].apply(get_lat_long))
# grouped_data.to_csv('album_winner_grammy_countries.csv', index=False)

In [54]:
#figure on grammy album winners from different countries
import geopandas as gpd
from shapely.geometry import Point
grouped_data=pd.read_csv('./dataset/album_winner_grammy_countries.csv')
geometry = [Point(xy) for xy in zip(grouped_data['longitude'], grouped_data['latitude'])]
geo_df = gpd.GeoDataFrame(grouped_data, geometry=geometry)
#world plot
fig_6 = px.scatter_geo(geo_df,
                    lat=geo_df.geometry.y,
                    lon=geo_df.geometry.x,
                    color = 'count',
                    hover_name="City/town of birth/origin", animation_frame='decade',
                    title='Grammy Album Winners from the World Decadewise')

fig_6.update_layout(
    # height=400,
    title_font=dict(size=25),
    height=600,
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)
fig_6.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500

fig_6.show()

#usa plot

In [55]:
#usa plot
fig_7 = px.scatter_geo(geo_df,
                    lat=geo_df.geometry.y,
                    lon=geo_df.geometry.x,
                    color = 'count',
                    hover_name="City/town of birth/origin", animation_frame='decade',
                    scope='usa',
                    title='Grammy Album Winners from the USA Decadewise')

fig_7.update_layout(
    # height=400,
    title_font=dict(size=25),
    height=600,
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)
fig_7.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500

fig_7.show()

In [46]:
#comparison of different formats of music - Revenue Generated
#Line chart
us_music_revenue = pd.read_csv('./dataset/US Recorded Music Revenues by Format.csv')

selected_df1 = us_music_revenue[us_music_revenue['Format'].isin(['Cassette', 'CD', 'DVD Audio', 'LP/EP', 'Music Video (Physical)', 'Vinyl Single'])]

selected_df2 = us_music_revenue[us_music_revenue['Format'].isin(['Download Album', 'Download Single', 'On-Demand Streaming (Ad-Supported)', 'Other Ad-Supported Streaming', 'Other Digital'])]

fig_8 = px.line(selected_df1, x="Year", y="Revenue (Inflation Adjusted)", color="Format",
              title="Inflation Adjusted USA Music Revenues By Different Physical Formats (2019)")

# customize the layout of the top 5 line chart
fig_8.update_layout(
    xaxis_title="Year",
    yaxis_title="Inflation Adjusted Revenue (2019)(US$)",
    font=dict(
        family="Courier New, monospace",
        size=13,
        color="#7f7f7f"
    ))

fig_8.update_layout(height = 400)

In [47]:
fig_9 = px.line(selected_df2, x="Year", y="Revenue (Inflation Adjusted)", color="Format",
              title="Inflation Adjusted USA Music Revenues By Different Digital Formats (2019)")

# customize the layout of the top 5 line chart
fig_9.update_layout(
    xaxis_title="Year",
    yaxis_title="Inflation Adjusted Revenue (2019)(US$)",
    font=dict(
        family="Courier New, monospace",
        size=13,
        color="#7f7f7f"
    ))

fig_9.update_layout(height = 400)

In [48]:
#top 10 grossing years for an artist billboard
df=pd.read_csv("./dataset/money-makers-bb.csv")
top_10_artists = df.nlargest(11, 'total')
top_10_artists['Artist']=top_10_artists['Artist']+'  ('+top_10_artists['year'].astype(str)+')'
top_10_artists['publishing']=top_10_artists['publishing'].astype(float)
top_10_artists['sales']=top_10_artists['sales'].astype(float)
top_10_artists['streaming']=top_10_artists['streaming'].astype(float)
top_10_artists['touring']=top_10_artists['touring'].astype(float)
fig_10 = px.bar(top_10_artists, x="Artist", y=["touring", "streaming", "publishing", "sales"], title="Top 10 Highest Grossing Years for an Artist (Billboard 2022 Data)", hover_data='year')
fig_10.update_layout(
    xaxis_title="Artist Name",
    yaxis_title="Total Earnings in a Year (Millions US$)",
    height=600)
fig_10.show()

In [49]:
#application integration
from dash import Dash, dcc, html, Input, Output
import base64

In [50]:

external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]

with open("./dataset/music_first image.jpg", "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

In [68]:
app = Dash(__name__,external_stylesheets =external_stylesheets)
styles = {
    'dropdown': {
        'width': '420px',
       ' padding-left':'20px',
        'display': 'inline-block',
        'font-size': '15px','border-color': '#2A547E','border-width': '2px',
        
    },
    'container': {
        'display': 'flex',
        'width': '420px',
        ' margin-left':'20px',
    }
}
app.layout = html.Div([
    html.Div(
        [   # Earth's image in the first line
    html.Img(src=f"data:image/jpg;base64,{encoded_image}", style={"height": "300px", "display": "block", "margin": "auto"}, ),
            
            html.H1("MUSIC EVOLUTION ANALYTICS", style={"text-align": "center","font-family":"PT Sans Narrow",'font-size': '45px' }),
            # Track changes in Earth's temperature, carbon emissions & Sea Levels over time in the third line
            html.H4("Analysis of Evolution of Commercial Music in the World", style={"text-align": "center","font-family":"PT Sans Narrow",'font-size': '20px'}),
        ],
        style={"padding-top": "10px", 'padding-bottom': '10px',"background-color": "black", "color": "white", 'box-shadow': '5px 5px 5px grey',"border-radius": "15px",}
    ),
    dcc.Dropdown(
            id="demo-dropdown",
            options=[
                {"label": "Global Music General Info", "value": "general_info"},
                {"label": "Music: Past vs Present", "value": "past_present"},
                {"label": "Case Studies", "value": "case_study"},
                {"label": "Spotify", "value": "spotify"}
            ],
            value="",
            placeholder="Select the Desired Visulization",
        style={'width':'450px',"margin-top": "20px", 'margin-bottom': '20px','padding-left': '20px','font-size': '15px','border-color': '#2A547E','border-width': '2px' }
        ),
    html.Div(id='dd-output-container'),
    
],        style={"background-color": "#CDDEEE", "padding": "10px"}

)


@app.callback(
    Output('dd-output-container', 'children'),
    Input('demo-dropdown', 'value')
)
def update_output(value):
    if value == 'general_info':
        return html.Div(
            children=[
                html.Div(children=[
                            dcc.Dropdown(
                                id='choro-dropdown',
                                options = [
                                    {'label': 'Pop', 'value': 'Pop'},
                                    {'label': 'EDM', 'value': 'EDM'},
                                    {'label': 'Rock/Metal', 'value': 'Rock/Metal'},
                                    {'label': 'Hip hop/Rap/R&b', 'value': 'Hip hop/Rap/R&b'}
                                ],
                                value='Pop',
                                 placeholder="Select a Genre to see Popularity",
                                style={'width':'450px',"margin-top": "20px", 'margin-bottom': '20px','padding-left': '20px','font-size': '15px','border-color': '#2A547E','border-width': '2px' }
                            ),
                            html.H3(children='Color Values represent the percentage of artists of a particular genre among popular artists in that Country', style={'font-size': '20px', 'color': 'black','padding-left':'10px'})
                        ]),
                dcc.Graph(id="fig_genre", figure=fig1_pop,style={"margin-bottom": "10px",'border': '3px solid #2A547E',}),

                html.H1(children='Top Albums of Different Generes over the years',
                            style={'font-size': '20px', 'color': 'black','padding-left':'10px'}),
                dcc.Graph(id='top_albums_genres', figure=fig5_genre_list,style={"margin-bottom": "10px",'border': '3px solid #2A547E'}),

                dcc.Graph(id='=grammy_world',figure=fig_6,style={"margin-bottom": "10px",'border': '3px solid #2A547E',}),

                dcc.Graph(id='grammy_usa',figure=fig_7,style={"margin-bottom": "10px",'border': '3px solid #2A547E'}),
                
                dcc.Graph(id='money_makers', figure=fig_10, style={"margin-bottom": "10px",'border': '3px solid #2A547E'})

        ],style={'background-color': '#4482C1'})
    


    elif value == 'past_present':
        return html.Div(
    
            children=[
                dcc.Graph(id='=revenue_physical',figure=fig_8,style={"margin-bottom": "10px",'border': '3px solid #2A547E',}),
                dcc.Graph(id='=revenue_digital',figure=fig_9,style={"margin-bottom": "10px",'border': '3px solid #2A547E',}),
            ],style={'background-color': '#4482C1'},         
        )
    
    elif value == 'case_study':
        return html.Div(
            children=[
            ],
            style={'background-color': '#4482C1'},

        )
    elif value == 'spotify':
        return html.Div(
            children=[

            ],style={'background-color': '#4482C1'})
    else:
        return html.Div()
    

#chloropleth genre dropdown update
curr_fig_genre=fig1_pop
@app.callback(Output('fig_genre', 'figure'),
[Input('choro-dropdown', 'value')])
def update_choro(value):
    global fig1_pop
    global fig2_edm
    global fig3_rock
    global fig4_hiphop
    global curr_fig_genre
    if value == 'Pop':
        curr_fig_genre=fig1_pop
        return fig1_pop
    elif value == 'EDM':
        curr_fig_genre=fig2_edm
        return fig2_edm
    elif value == 'Rock/Metal':
        curr_fig_genre=fig3_rock
        return fig3_rock
    elif value == 'Hip hop/Rap/R&b':
        curr_fig_genre=fig4_hiphop
        return fig4_hiphop
    else:
        return curr_fig_genre



In [69]:
if __name__ == '__main__':
    app.run_server()